# Retail Sample Data Generator

This notebook generates a sample retail database with the following tables:
- `location`: Store and depot locations
- `item`: Retail items with pricing
- `sales_header`: Sales transactions
- `sales_line`: Line items for each sale
- `daily_stock`: Daily stock snapshots

**Important**: All dates are stored as ISO 8601 text strings ("YYYY-MM-DD") following SQLite best practices.

In [4]:
import sqlite3
import random
from datetime import datetime, timedelta
import os

## Database Setup

In [5]:
# Use current working directory for database file
db_path = 'C:\\Users\\cw171001\\Projects\\prototype_chameleon\\chameleon.db'

## Remove existing database if it exists
#if os.path.exists(db_path):
 #   os.remove(db_path)
 #   print(f"Removed existing database: {db_path}")

# Connect to the database (will create it)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
print(f"Connected to database: {db_path}")

Connected to database: C:\Users\cw171001\Projects\prototype_chameleon\chameleon.db


## Table 1: Location

Stores and depots across Ireland.

In [6]:
# Create location table
# Store information for retail locations including stores and depots
cursor.execute('''
CREATE TABLE location (
    loc_id INTEGER PRIMARY KEY,
    loc_type TEXT NOT NULL CHECK(loc_type IN ('Store', 'Depot')),
    loc_name TEXT NOT NULL,
    loc_country TEXT NOT NULL,
    loc_town TEXT NOT NULL
)
''')

# Insert location data: 1 Depot + 10 Stores in different Irish towns
locations = [
    (1, 'Depot', 'Central Distribution Depot', 'Ireland', 'Dublin'),
    (2, 'Store', 'Cork City Store', 'Ireland', 'Cork'),
    (3, 'Store', 'Galway Shopping Centre', 'Ireland', 'Galway'),
    (4, 'Store', 'Limerick Main Street', 'Ireland', 'Limerick'),
    (5, 'Store', 'Waterford Quayside', 'Ireland', 'Waterford'),
    (6, 'Store', 'Drogheda Town Centre', 'Ireland', 'Drogheda'),
    (7, 'Store', 'Dundalk Retail Park', 'Ireland', 'Dundalk'),
    (8, 'Store', 'Bray Seafront', 'Ireland', 'Bray'),
    (9, 'Store', 'Navan Shopping Mall', 'Ireland', 'Navan'),
    (10, 'Store', 'Kilkenny High Street', 'Ireland', 'Kilkenny'),
    (11, 'Store', 'Tralee Town Centre', 'Ireland', 'Tralee')
]

cursor.executemany('INSERT INTO location VALUES (?, ?, ?, ?, ?)', locations)
print(f"Inserted {len(locations)} locations (1 depot, 10 stores)")
conn.commit()

Inserted 11 locations (1 depot, 10 stores)


## Table 2: Item

Retail items with pricing and categorization.

In [7]:
# Create item table
# Retail product catalog with pricing and categorization
cursor.execute('''
CREATE TABLE item (
    item_id INTEGER PRIMARY KEY,
    item_name TEXT NOT NULL,
    item_type TEXT NOT NULL,
    department TEXT NOT NULL,
    section TEXT NOT NULL,
    retail_price REAL NOT NULL,
    cost_price REAL NOT NULL,
    CHECK(retail_price > cost_price)
)
''')

# Insert 20 realistic retail items with proper margins
items = [
    (1, 'Cotton T-Shirt', 'Apparel', 'Mens', 'Casual Wear', 19.99, 8.00),
    (2, 'Denim Jeans', 'Apparel', 'Mens', 'Casual Wear', 59.99, 25.00),
    (3, 'Running Shoes', 'Footwear', 'Sports', 'Athletic', 89.99, 40.00),
    (4, 'Leather Wallet', 'Accessories', 'Mens', 'Leather Goods', 34.99, 15.00),
    (5, 'Wool Sweater', 'Apparel', 'Mens', 'Knitwear', 49.99, 22.00),
    (6, 'Summer Dress', 'Apparel', 'Womens', 'Dresses', 69.99, 30.00),
    (7, 'Handbag', 'Accessories', 'Womens', 'Bags', 79.99, 35.00),
    (8, 'High Heels', 'Footwear', 'Womens', 'Formal', 99.99, 45.00),
    (9, 'Scarf', 'Accessories', 'Womens', 'Winter', 24.99, 10.00),
    (10, 'Blouse', 'Apparel', 'Womens', 'Formal Wear', 39.99, 18.00),
    (11, 'Kids Sneakers', 'Footwear', 'Kids', 'Athletic', 44.99, 20.00),
    (12, 'School Backpack', 'Accessories', 'Kids', 'School', 29.99, 12.00),
    (13, 'Kids Hoodie', 'Apparel', 'Kids', 'Casual Wear', 34.99, 15.00),
    (14, 'Baseball Cap', 'Accessories', 'Mens', 'Headwear', 19.99, 8.00),
    (15, 'Sports Socks Pack', 'Apparel', 'Sports', 'Athletic', 14.99, 6.00),
    (16, 'Winter Jacket', 'Apparel', 'Mens', 'Outerwear', 129.99, 60.00),
    (17, 'Sunglasses', 'Accessories', 'Unisex', 'Eyewear', 54.99, 22.00),
    (18, 'Belt', 'Accessories', 'Mens', 'Leather Goods', 29.99, 12.00),
    (19, 'Cardigan', 'Apparel', 'Womens', 'Knitwear', 54.99, 24.00),
    (20, 'Polo Shirt', 'Apparel', 'Mens', 'Casual Wear', 39.99, 16.00)
]

cursor.executemany('INSERT INTO item VALUES (?, ?, ?, ?, ?, ?, ?)', items)
print(f"Inserted {len(items)} retail items")
conn.commit()

Inserted 20 retail items


## Table 3: Sales Header

Sales transactions for the last 30 days with 20-50 sales per store per day.

**Note**: `sales_date` is stored as ISO 8601 text string ("YYYY-MM-DD").

In [8]:
# Create sales_header table
# Transaction headers - dates stored as ISO 8601 text strings (YYYY-MM-DD)
cursor.execute('''
CREATE TABLE sales_header (
    sale_id INTEGER PRIMARY KEY,
    loc_id INTEGER NOT NULL,
    sales_date TEXT NOT NULL,
    tender_total REAL NOT NULL,
    FOREIGN KEY (loc_id) REFERENCES location(loc_id)
)
''')

# Generate sales for last 30 days
# Only stores (not depot) generate sales
store_ids = [loc[0] for loc in locations if loc[1] == 'Store']
end_date = datetime.now()
start_date = end_date - timedelta(days=29)  # Last 30 days inclusive

sale_id = 1
sales_headers = []

# Iterate through each day
current_date = start_date
while current_date <= end_date:
    # Format date as ISO 8601 string (YYYY-MM-DD)
    date_str = current_date.strftime('%Y-%m-%d')
    
    # For each store
    for store_id in store_ids:
        # Generate 20-50 sales per day per store
        num_sales = random.randint(20, 50)
        
        for _ in range(num_sales):
            # Placeholder for tender_total (will be updated after sales_line)
            sales_headers.append((sale_id, store_id, date_str, 0.0))
            sale_id += 1
    
    current_date += timedelta(days=1)

cursor.executemany('INSERT INTO sales_header VALUES (?, ?, ?, ?)', sales_headers)
print(f"Inserted {len(sales_headers)} sales headers (30 days, 10 stores, 20-50 sales/day)")
conn.commit()

Inserted 10376 sales headers (30 days, 10 stores, 20-50 sales/day)


## Table 4: Sales Line

Line items for each sale with 1-5 random items per sale.

In [9]:
# Create sales_line table
# Individual line items for each transaction
cursor.execute('''
CREATE TABLE sales_line (
    sale_id INTEGER NOT NULL,
    item_id INTEGER NOT NULL,
    sales_amount REAL NOT NULL,
    sales_units INTEGER NOT NULL,
    FOREIGN KEY (sale_id) REFERENCES sales_header(sale_id),
    FOREIGN KEY (item_id) REFERENCES item(item_id),
    PRIMARY KEY (sale_id, item_id)
)
''')

# Create a dictionary of item prices for quick lookup
item_prices = {item[0]: item[5] for item in items}  # item_id: retail_price

# Generate sales lines and calculate tender totals
sales_lines = []
tender_totals = {}  # sale_id: total_amount

for sale_header in sales_headers:
    sale_id = sale_header[0]
    
    # 1-5 items per sale
    num_items = random.randint(1, 5)
    selected_items = random.sample(list(item_prices.keys()), num_items)
    
    sale_total = 0.0
    
    for item_id in selected_items:
        # 1-3 units per item
        units = random.randint(1, 3)
        retail_price = item_prices[item_id]
        amount = units * retail_price
        
        sales_lines.append((sale_id, item_id, amount, units))
        sale_total += amount
    
    tender_totals[sale_id] = sale_total

cursor.executemany('INSERT INTO sales_line VALUES (?, ?, ?, ?)', sales_lines)
print(f"Inserted {len(sales_lines)} sales line items")

# Update tender_total in sales_header
for sale_id, total in tender_totals.items():
    cursor.execute('UPDATE sales_header SET tender_total = ? WHERE sale_id = ?', (total, sale_id))

print("Updated tender_total for all sales headers")
conn.commit()

Inserted 30975 sales line items
Updated tender_total for all sales headers


## Table 5: Daily Stock

Daily stock snapshots for all locations and items over the last 30 days.

**Note**: `stock_date` is stored as ISO 8601 text string ("YYYY-MM-DD").

In [10]:
# Create daily_stock table
# Daily stock snapshots - dates stored as ISO 8601 text strings (YYYY-MM-DD)
cursor.execute('''
CREATE TABLE daily_stock (
    stock_date TEXT NOT NULL,
    loc_id INTEGER NOT NULL,
    item_id INTEGER NOT NULL,
    stock_on_hand_units INTEGER NOT NULL,
    stock_on_order_units INTEGER NOT NULL,
    FOREIGN KEY (loc_id) REFERENCES location(loc_id),
    FOREIGN KEY (item_id) REFERENCES item(item_id),
    PRIMARY KEY (stock_date, loc_id, item_id)
)
''')

# Generate daily stock for all locations (including depot) and all items
all_loc_ids = [loc[0] for loc in locations]
all_item_ids = [item[0] for item in items]

daily_stock_records = []

# Iterate through each day
current_date = start_date
while current_date <= end_date:
    # Format date as ISO 8601 string (YYYY-MM-DD)
    date_str = current_date.strftime('%Y-%m-%d')
    
    # For each location
    for loc_id in all_loc_ids:
        # For each item
        for item_id in all_item_ids:
            # Random stock levels
            # Depots typically have more stock than stores
            if loc_id == 1:  # Depot
                stock_on_hand = random.randint(100, 500)
                stock_on_order = random.randint(0, 200)
            else:  # Store
                stock_on_hand = random.randint(5, 50)
                stock_on_order = random.randint(0, 30)
            
            daily_stock_records.append((date_str, loc_id, item_id, stock_on_hand, stock_on_order))
    
    current_date += timedelta(days=1)

cursor.executemany('INSERT INTO daily_stock VALUES (?, ?, ?, ?, ?)', daily_stock_records)
print(f"Inserted {len(daily_stock_records)} daily stock records (30 days × 11 locations × 20 items)")
conn.commit()

Inserted 6600 daily stock records (30 days × 11 locations × 20 items)


## Verification

Let's verify the database has been populated correctly.

In [11]:
# Verify table counts
tables = ['location', 'item', 'sales_header', 'sales_line', 'daily_stock']

print("\nDatabase Summary:")
print("=" * 50)

for table in tables:
    cursor.execute(f'SELECT COUNT(*) FROM {table}')
    count = cursor.fetchone()[0]
    print(f"{table:20s}: {count:,} records")

# Show sample data from each table
print("\n\nSample Data:")
print("=" * 50)

print("\nLocations (first 5):")
cursor.execute('SELECT * FROM location LIMIT 5')
for row in cursor.fetchall():
    print(row)

print("\nItems (first 5):")
cursor.execute('SELECT item_id, item_name, department, retail_price, cost_price FROM item LIMIT 5')
for row in cursor.fetchall():
    print(row)

print("\nSales Headers (first 5):")
cursor.execute('SELECT * FROM sales_header LIMIT 5')
for row in cursor.fetchall():
    print(row)

print("\nSales Lines (first 5):")
cursor.execute('SELECT * FROM sales_line LIMIT 5')
for row in cursor.fetchall():
    print(row)

print("\nDaily Stock (first 5):")
cursor.execute('SELECT * FROM daily_stock LIMIT 5')
for row in cursor.fetchall():
    print(row)

# Verify date format
print("\n\nDate Format Verification:")
print("=" * 50)
cursor.execute('SELECT DISTINCT sales_date FROM sales_header ORDER BY sales_date LIMIT 5')
print("Sample sales_date values (should be YYYY-MM-DD):")
for row in cursor.fetchall():
    print(f"  {row[0]}")

cursor.execute('SELECT DISTINCT stock_date FROM daily_stock ORDER BY stock_date LIMIT 5')
print("\nSample stock_date values (should be YYYY-MM-DD):")
for row in cursor.fetchall():
    print(f"  {row[0]}")


Database Summary:
location            : 11 records
item                : 20 records
sales_header        : 10,376 records
sales_line          : 30,975 records
daily_stock         : 6,600 records


Sample Data:

Locations (first 5):
(1, 'Depot', 'Central Distribution Depot', 'Ireland', 'Dublin')
(2, 'Store', 'Cork City Store', 'Ireland', 'Cork')
(3, 'Store', 'Galway Shopping Centre', 'Ireland', 'Galway')
(4, 'Store', 'Limerick Main Street', 'Ireland', 'Limerick')
(5, 'Store', 'Waterford Quayside', 'Ireland', 'Waterford')

Items (first 5):
(1, 'Cotton T-Shirt', 'Mens', 19.99, 8.0)
(2, 'Denim Jeans', 'Mens', 59.99, 25.0)
(3, 'Running Shoes', 'Sports', 89.99, 40.0)
(4, 'Leather Wallet', 'Mens', 34.99, 15.0)
(5, 'Wool Sweater', 'Mens', 49.99, 22.0)

Sales Headers (first 5):
(1, 2, '2025-11-13', 69.99)
(2, 2, '2025-11-13', 79.98)
(3, 2, '2025-11-13', 239.94)
(4, 2, '2025-11-13', 419.9)
(5, 2, '2025-11-13', 704.8900000000001)

Sales Lines (first 5):
(1, 6, 69.99, 1)
(2, 20, 79.98, 2)
(3, 7, 7

## Close Connection

In [ ]:
# Close the database connection
conn.close()
print(f"\nDatabase created successfully: {db_path}")
print(f"Total file size: {os.path.getsize(db_path):,} bytes")